In [1]:
import numpy as np

# Intro to PyConGrad

PyConGrad is a backend-agnostic conjugate gradient implementation.  For this tutorial we'll be using NumPy, but we could switch to CuPy or PyTorch in just a couple of lines or write a custom backend for almost any vector implementation that supports NumPy-like broadcasting and standard `+-*/` operator overloading.

In [2]:
N = 100
X = np.random.rand(N, N)
A = X @ X.T + 0.001 * np.eye(100) # Guarantee that A is SPD
batches = 2
b = np.random.rand(batches, N)

PyConGrad does not actually require explicitly-formed matrices.  It takes both its matvec and its (optional) preconditioner argument as functions that act on batched right-hand sides.  There's no PyConGrad reason why you can't have arbitrarily many batch dimensions; here we'll just use one.  The last dimension is always the vector dimension.

In [3]:
def A_batch(b):
    return np.einsum("ij,bj->bi", A, b)

def diagonal_preconditioner(b):
    return np.expand_dims(1 / np.diag(A), 0) * b

Now we import `cg_batch` from the relevant backend.  For the purposes of documentation, we will specify every possible option here.  Mostly, you can use the defaults.

In [4]:
from congrad.numpy import cg_batch

In [5]:
solution, solve_info = cg_batch(A_batch,
                                b,
                                P=diagonal_preconditioner, # Preconditioner (default: None)
                                x0=None,                   # Initial guess (default: None; will default to b in that case)
                                rtol=1e-9,                 # Relative error tolerance, relative to |b| (default: 1e-3)
                                atol=0,                    # Absolute error tolerance (default: 0)
                                maxiter=1000,              # Maximum number of iterations, or None to have no maximum (default: 1000)
                                warn_unconverged=True,     # Raise a warning if convergence fails (default: True)
                                monitor=True,              # True to use the default monitor for solve progress, which prints the residual every 20 iters;
                                                           # False or None to not use any monitor; or a Monitor subclass to use that monitor in particular.
                                                           # (Default: None)
                                flexible=False)            # Use "flexible CG" (Polak-Ribère rather than Fletcher-Reeves) (default: False)

020: 2.16268e+00 (4.83966e-03 seconds)
040: 1.88205e+00 (7.93743e-03 seconds)
060: 1.31465e+01 (1.09115e-02 seconds)
080: 1.99240e+00 (1.34926e-02 seconds)
100: 1.52766e+00 (1.48268e-02 seconds)
120: 9.47478e-01 (1.55027e-02 seconds)
140: 4.29007e-01 (1.62010e-02 seconds)
160: 1.62021e-01 (1.69029e-02 seconds)
180: 9.59698e-06 (1.74885e-02 seconds)
Finished in 1.77190e-02 seconds after 187 iterations (1.05536e+04 iterations/second) with a maximum residual of 5.25773e-09.


In [6]:
for batch in range(batches):
    print(np.linalg.norm(A @ solution[batch] - b[batch]) / np.linalg.norm(b[batch]))

3.3359699583984326e-09
4.910245252734883e-10


In [7]:
solve_info

{'niter': 187,
 'converged': True,
 'residual': array([5.25773053e-09, 1.26562434e-09])}

# Custom Backends

If we want to use a new vector backend, we can extend the `Backend` class to enable PyConGrad to talk to it.  For the purposes of the tutorial, we'll pretend we don't have a NumPy backend yet.

All the methods we write here are defined as [static](https://docs.python.org/3/library/functions.html#staticmethod) in the base `Backend` class, so they don't take a `self` argument.

In [8]:
from congrad import Backend, cg_batch_generic

In [9]:
class NumPyBackend(Backend):
    def norm(X): # Norm in vector dimension
        return np.linalg.norm(X, axis=-1)
    
    def dot(X, Y): # Dot product in vector dimension
        XX = np.expand_dims(X, -2)
        YY = np.expand_dims(Y, -1)
        dot_prod = np.matmul(XX, YY)
        return np.squeeze(dot_prod, axis=-1)
            
    def all_true(X): # Is every element of this boolean tensor true?  (Does not respect batching.)
        return X.all()
    
    def max_vector_scalar(X, y): # Elementwise maximum of a tensor and a scalar.  (Does not respect batching.)
        return np.maximum(X, y)
    
    def presentable_norm(residual): # Optional function that will be used to process residual norms before passing them to the monitor.
                                    # Has no effect on the solve itself (unless you're using your monitor to alter the solve on the fly).
        return np.max(residual).item()

Now that we have a backend, we create a CG function by using the class (not an instantiated object, since all the methods are static) to initialize a `cg_batch_generic` instance.  (Technically, `cg_batch_generic` is a class that implements `__call__`, meaning it can be called like a function.  In practice, you can think of `cg_batch_generic` as a function that maps backends to CG implementations.)

In [10]:
cg_batch_numpy = cg_batch_generic(NumPyBackend)

This can then be used just like any other `cg_batch` function.

In [11]:
solution, solve_info = cg_batch_numpy(A_batch, b, monitor=True)

020: 2.66609e+00 (1.58596e-03 seconds)
040: 2.22569e+00 (2.70104e-03 seconds)
060: 3.03677e+00 (4.16160e-03 seconds)
080: 1.96987e+00 (5.50532e-03 seconds)
100: 2.35592e+00 (6.46019e-03 seconds)
120: 5.37626e+00 (7.10320e-03 seconds)
140: 2.48394e-01 (7.84230e-03 seconds)
160: 2.21951e-01 (8.53968e-03 seconds)
Finished in 9.24087e-03 seconds after 176 iterations (1.90458e+04 iterations/second) with a maximum residual of 5.28046e-03.
